In [13]:
import os
import re
import json
import csv
from collections import defaultdict, Counter

ROOT_DIR = "/scratch/doluk/Compact-Interference-PRAG/PRAG/warmup/output/freezeA/warmup_cot"
OUT_DIR = "metrics_by_rank_epoch"
REQUIRED_PREDICTIONS = 300

os.makedirs(OUT_DIR, exist_ok=True)

# Regex
RANK_ALPHA_RE = re.compile(r"rank=(\d+)_alpha=(\d+)")
LR_EPOCH_RE = re.compile(r"lr=([\d.]+)_epoch=(\d+)")

# -------------------------
# Helpers
# -------------------------

def parse_result_txt(path):
    metrics = {"EM": "", "F1": "", "PREC": "", "RECALL": ""}
    with open(path) as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split()
            if len(parts) < 2:
                continue
            key = parts[0].lower()
            if key == "em":
                metrics["EM"] = parts[1]
            elif key == "f1":
                metrics["F1"] = parts[1]
            elif key == "prec":
                metrics["PREC"] = parts[1]
            elif key == "recall":
                metrics["RECALL"] = parts[1]
    return metrics

def valid_predictions(path):
    try:
        with open(path) as f:
            data = json.load(f)
        if isinstance(data, list) and len(data) == REQUIRED_PREDICTIONS:
            return True
        else:
            print(f"[WARN] Unfinished prediction for {path}")
            return False
    except Exception:
        return False

def infer_subset(parts):
    """
    Dataset-agnostic subset inference.
    Handles 2Wiki compound subsets.
    """
    for p in reversed(parts):
        name = p.lower()
        if "bridge_comparison" in name:
            return "bridge_comp"
        if "compositional" in name:
            return "compositional"
        if "comparison" in name:
            return "comparison"
        if "inference" in name:
            return "inference"
        if name == "total":
            return "total"
        if "bridge" in name:
            return "bridge"
    return "total"

def infer_method(parts):
    for p in parts:
        if p in {"prag", "combine"}:
            return p
    return ""

# -------------------------
# Collect experiments
# -------------------------

groups = defaultdict(list)

for root, _, files in os.walk(ROOT_DIR):
    if "result.txt" not in files:
        continue

    result_path = os.path.join(root, "result.txt")
    predict_path = os.path.join(root, "predict.json")

    parts = result_path.split(os.sep)

    metrics = parse_result_txt(result_path)
    pred_ok = valid_predictions(predict_path)

    dataset = rank = epoch = None

    for i, p in enumerate(parts):
        m = RANK_ALPHA_RE.search(p)
        if m:
            rank = int(m.group(1))

        m = LR_EPOCH_RE.search(p)
        if m:
            epoch = int(m.group(2))

        if RANK_ALPHA_RE.match(p) and i + 1 < len(parts):
            dataset = parts[i + 1]

    if rank is None or epoch is None or dataset is None:
        continue

    subset = infer_subset(parts)
    method = infer_method(parts)

    rel_result_path = os.path.relpath(result_path, ROOT_DIR)

    groups[(rank, epoch)].append({
        "label": f"{dataset}/{subset}/{method}",
        "result_path": rel_result_path,
        "metrics": metrics,
        "pred_check": "" if pred_ok else "!",
    })

# -------------------------
# Write one CSV per group
# -------------------------

for (rank, epoch), experiments in sorted(groups.items()):

    # -------- ASSERTION: labels must be unique --------
    labels = [e["label"] for e in experiments]
    dupes = [lbl for lbl, c in Counter(labels).items() if c > 1]

    assert not dupes, (
        f"[ERROR] Duplicate labels in rank={rank}, epoch={epoch}: {dupes}\n"
        f"Check subset inference or directory structure."
    )
    # -------------------------------------------------

    out_csv = os.path.join(OUT_DIR, f"metrics_r{rank}_eps{epoch}.csv")

    with open(out_csv, "w", newline="") as f:
        writer = csv.writer(f)

        # Header row 1: logical labels
        writer.writerow([""] + [e["label"] for e in experiments])

        # Header row 2: exact result paths
        # writer.writerow(["result_path"] + [e["result_path"] for e in experiments])

        # Metric rows
        for metric in ["EM", "F1", "PREC", "RECALL"]:
            writer.writerow(
                [metric] +
                [
                    e["metrics"][metric] + e["pred_check"]
                    for e in experiments
                ]
            )

    print(f"Saved → {out_csv}")


[WARN] Unfinished prediction for /scratch/doluk/Compact-Interference-PRAG/PRAG/warmup/output/freezeA/warmup_cot/qwen2.5-1.5b-instruct/rank=2_alpha=32/popqa/lr=0.0003_epoch=1_direct/aug_model=qwen2.5-1.5b-instruct/prag/total/predict.json
Saved → metrics_by_rank_epoch/metrics_r2_eps1.csv
Saved → metrics_by_rank_epoch/metrics_r2_eps2.csv
Saved → metrics_by_rank_epoch/metrics_r2_eps4.csv
Saved → metrics_by_rank_epoch/metrics_r4_eps2.csv
Saved → metrics_by_rank_epoch/metrics_r4_eps4.csv
Saved → metrics_by_rank_epoch/metrics_r8_eps2.csv
Saved → metrics_by_rank_epoch/metrics_r8_eps4.csv
Saved → metrics_by_rank_epoch/metrics_r16_eps2.csv
Saved → metrics_by_rank_epoch/metrics_r16_eps4.csv


In [2]:
import re
from pathlib import Path

def parse_result_txt(result_path: Path):
    text = result_path.read_text()
    results = {}
    current_epoch = None

    for line in text.splitlines():
        line = line.strip()
        if not line:
            continue

        # Epoch line
        m_epoch = re.match(r"Epoch\s+(\d+)", line)
        if m_epoch:
            current_epoch = int(m_epoch.group(1))
            results[current_epoch] = {}
            continue

        # Metric line
        m_metric = re.match(r"(em|f1|prec|recall)\s+([0-9.]+)", line)
        if m_metric and current_epoch is not None:
            metric, value = m_metric.groups()
            results[current_epoch][metric] = float(value)

    return results
from pathlib import Path

full_results = {}
for lora_rank in [2, 4, 8, 16]:
    for epoch in [2, 4]:
        result_path = Path(
            "/scratch/doluk/Compact-Interference-PRAG/output/moe/qwen2.5-1.5b-instruct/"
            f"rank={lora_rank}_alpha=32/popqa/lr=0.0003_epoch={epoch}_direct/"
            "aug_model=qwen2.5-1.5b-instruct/post_train_epochs=20/"
            "lr=0.0003_direct/prag/total/result.txt"
        )
        try:
            results = parse_result_txt(result_path)
            full_results[(lora_rank, epoch)]=results
        except FileNotFoundError as e:
            print(e)

[Errno 2] No such file or directory: '/scratch/doluk/Compact-Interference-PRAG/output/moe/qwen2.5-1.5b-instruct/rank=16_alpha=32/popqa/lr=0.0003_epoch=4_direct/aug_model=qwen2.5-1.5b-instruct/post_train_epochs=20/lr=0.0003_direct/prag/total/result.txt'


In [3]:
F1 = {}
for key, value in full_results.items():
    f1 = [0]*20
    for epoch, metrics in value.items():
        f1[epoch] = metrics["f1"]
    F1[key] = f1

In [5]:
import pandas as pd
df = pd.DataFrame(F1)

df.columns = pd.MultiIndex.from_tuples(
    df.columns,
    names=["rank", "train_epoch"]
)

df.index.name = "post_epoch"
df.to_csv("results/qwen2.5-1.5b-instruct/moe/popqa/total.csv")

In [7]:
import pandas as pd
from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import Font

# Build DataFrame
df = pd.DataFrame(F1)

df.columns = pd.MultiIndex.from_tuples(
    df.columns,
    names=["rank", "train_epoch"]
)
df.index.name = "post_epoch"

# Output path
out_path = Path("results/qwen2.5-1.5b-instruct/moe/popqa/total.xlsx")
out_path.parent.mkdir(parents=True, exist_ok=True)

# Write DataFrame
df.to_excel(out_path)

# Open workbook for formatting
wb = load_workbook(out_path)
ws = wb.active
bold = Font(bold=True)

# ---- IMPORTANT: exact data region ----
DATA_START_ROW = 3   # first metric row
DATA_START_COL = 2   # first metric column

n_rows, n_cols = df.shape

for col_idx in range(n_cols):
    col_values = df.iloc[:, col_idx]
    max_val = col_values.max()

    for row_idx, val in enumerate(col_values):
        if val == max_val:
            ws.cell(
                row=DATA_START_ROW + row_idx,
                column=DATA_START_COL + col_idx
            ).font = bold

wb.save(out_path)
